<!--
tags: [tutorial]
description: Human-in-the-loop
-->

# Allowing Human Feedback and Termination of Conversations

In the [last chapter](./introduction) we introduced the `ConversableAgent` class.
We also briefly mentioned that this
class supports a human-in-the-loop component.
In this chapter, we will show how you can use this component to both allow human 
feedback to an agent and also control when to terminate a conversation between agents.

The figure below show an overview of the human-in-the-loop component and how it is used.

````mdx-code-block
![human-in-the-loop-component](./assets/human-in-the-loop.png)
````

As shown in the figure, when an agent receives a message, it is first intercepted by
the human-in-the-loop component, which decides between the following options:

1. pass the message to the other components that generates an automatic response
2. _short-circuit_ other components and ask for and return a response from a human
3. terminate the conversation

## Termination Conditions

By default, the human-in-the-loop component considers two conditions for deciding whether to terminate a conversation:

1. _Message-based_: This condition triggers termination if the content of a message contains
    the word "TERMINATE". You can customize this condition
    using the `is_terminate_msg` argument in the constructor of the `ConversableAgent` class.
2. _Counter-based_: This condition trigger termination if the number of automatic responses to the same sender
    exceeds a threshold. You can customize this using the `max_consecutive_auto_reply` argument of the
    `ConversableAgent` class.

It is important to note that when a termination condition is triggered,
the conversation is not always terminated immediately. The actual termination
depends on the `human_input_mode` argument of the `ConversableAgent` class.

Here is an example that alters the default message-based termination condition.
In this example, the termination condition is triggered if the message
contains the phrase _"good bye my friend"_.

In [10]:
import os
from autogen import ConversableAgent

term_by_msg_agent = ConversableAgent(
    "auto_reply_agent",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    is_termination_msg=lambda msg: "good bye my friend" in msg["content"].lower(),
)

Here is a different example for that uses the counter-based termination condition.
In this example, the termination condition is triggered if the number of
automatic responses to the same sender exceeds 3.

In [11]:
term_by_count_agent = ConversableAgent(
    "auto_reply_agent",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    max_consecutive_auto_reply=3,
)

## Human Input Modes

Currently AutoGen supports three modes for human input. The mode is specified through
the `human_input_mode` argument of the `ConversableAgent`. And these modes combined
with the termination conditions dictate how the human-in-the-loop component
behaves:

1. `NEVER`: human input is never requested, and the termination conditions are
    used determine when to terminate.
2. `ALWAYS`: human input is always requested and the human can choose to skip,
    intercept, or terminate the conversation; message-based termination
    condition is still enacted upon if human chooses to skip; counter-based
    termination condition is ignored.
3. `TERMINATE`: human input is only requested when a termination condition is
    met -- this is the default mode; if the human chooses to skip, automatic
    reply components are used; 
    if the human chooses to intercept and reply, the conversation continues
    and the counter is reset; if the human chooses to terminate, the conversation
    is then terminated.

Let us see each of these modes in action.

### Human Input Mode = `NEVER`

In this mode, human input is never requested and the termination conditions
are used to terminate.
This mode is useful only when you want your agents to run fully autonomously.

Here is an example of using this mode to run a simple guess-a-number game between
two agents, with the maximum number of guesses set to 5 through
counter-based termination, and the termination message set to check for the 
number that is the correct guess.

In [12]:
agent_with_number = ConversableAgent(
    "agent_with_number",
    system_message="You are playing a game of guess-my-number. You have the "
    "number 53 in your mind, and I will try to guess it. "
    "If I guess too high, say 'too high', if I guess too low, say 'too low'. ",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    is_termination_msg=lambda msg: "53" in msg["content"],  # terminate if the number is guessed by the other agent
    max_consecutive_auto_reply=5,  # set maximum turn-based auto-reply to 5
    human_input_mode="NEVER",  # never ask for human input
)

agent_guess_number = ConversableAgent(
    "agent_guess_number",
    system_message="I have a number in my mind, and you will try to guess it. "
    "If I say 'too high', you should guess a lower number. If I say 'too low', "
    "you should guess a higher number. ",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

result = agent_with_number.initiate_chat(
    agent_guess_number,
    message="I have a number between 1 and 100. Guess it!",
)

agent_with_number (to agent_guess_number):

I have a number between 1 and 100. Guess it!

--------------------------------------------------------------------------------
agent_guess_number (to agent_with_number):

50

--------------------------------------------------------------------------------
agent_with_number (to agent_guess_number):

Too low.

--------------------------------------------------------------------------------
agent_guess_number (to agent_with_number):

75

--------------------------------------------------------------------------------
agent_with_number (to agent_guess_number):

Too high.

--------------------------------------------------------------------------------
agent_guess_number (to agent_with_number):

62

--------------------------------------------------------------------------------
agent_with_number (to agent_guess_number):

Too high.

--------------------------------------------------------------------------------
agent_guess_number (to agent_with_n

Yay! The game is over. The guessing agent got the number correctly in exactly
5 guesses using binary search -- very efficient!
You can see that the conversation was terminated after the 5th guess which
triggered both the counter-based and message-based termination conditions.

### Human Input Mode = `ALWAYS`

In this mode, human input is always requested and the human can choose to skip,
intersecpt, or terminate the conversation.
Let us see this mode in action by playing the same game as before with the agent with the number, but this time
participating in the game as a human.
We will be the agent that is guessing the number, and play against the agent
with the number from before.

In [14]:
human_proxy = ConversableAgent(
    "human_proxy",
    llm_config=False,  # no LLM used for human proxy
    human_input_mode="ALWAYS",  # always ask for human input
)

# Start a chat with the agent with number with an initial guess.
result = human_proxy.initiate_chat(
    agent_with_number,  # this is the same agent with the number as before
    message="10",
)

human_proxy (to agent_with_number):

10

--------------------------------------------------------------------------------
agent_with_number (to human_proxy):

Too low.

--------------------------------------------------------------------------------
human_proxy (to agent_with_number):

75

--------------------------------------------------------------------------------
agent_with_number (to human_proxy):

Too high.

--------------------------------------------------------------------------------
human_proxy (to agent_with_number):

I give up

--------------------------------------------------------------------------------
agent_with_number (to human_proxy):

The number I was thinking of was 53. Give it another try next time, it's all about making educated guesses and having fun!

--------------------------------------------------------------------------------


If you run the code above, you will be prompt to enter a response
each time it is your turn to speak. You can see the human in the conversation
was not very good at guessing the number... but hey the agent was nice enough
to give out the number in the end.

### Human Input Mode = `TERMINATE`

In this mode, human input is only requested when a termination condition is
met. **If the human choose to intercept and reply, counter will be rest**; if 
the human choose to skip, automatic reply mechanism will be used; if the human
choose to terminate, the conversation will be terminated.

Let us see this mode in action by playing the same game again, but this time
the agent with the number will choose a new number and the game will restart.

In [15]:
agent_with_number = ConversableAgent(
    "agent_with_number",
    system_message="You are playing a game of guess-my-number. "
    "In the first game, you have the "
    "number 53 in your mind, and I will try to guess it. "
    "If I guess too high, say 'too high', if I guess too low, say 'too low'. ",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    max_consecutive_auto_reply=5,
    human_input_mode="TERMINATE",  # ask for human input until the game is terminated
)

agent_guess_number = ConversableAgent(
    "agent_guess_number",
    system_message="I have a number in my mind, and you will try to guess it. "
    "If I say 'too high', you should guess a lower number. If I say 'too low', "
    "you should guess a higher number. ",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

result = agent_with_number.initiate_chat(
    agent_guess_number,
    message="I have a number between 1 and 100. Guess it!",
)

agent_with_number (to agent_guess_number):

I have a number between 1 and 100. Guess it!

--------------------------------------------------------------------------------
agent_guess_number (to agent_with_number):

50

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent_with_number (to agent_guess_number):

Too low.

--------------------------------------------------------------------------------
agent_guess_number (to agent_with_number):

75

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent_with_number (to agent_guess_number):

Too high.

--------------------------------------------------------------------------------
agent_guess_number (to agent_with_number):

62

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent_with_number (to agent_guess_number):

Too high.

--------------------------

In the previous conversation, we were asked to provide human input first time
when the counter-based termination condition was met. We intercepted the
message and replied "Let's play again". The conversation continued and the
counter was reset. The game continued until the counter-based termination
condition was met again, and this time we chose to skip. The conversation
continued again but human input was requested at every turn since, and we
chose to skip each time until the game was over.

## Summary

In this chapter, we showed you how to use the human-in-the-loop component
to provide human feedback to agent and to terminate conversation.
We also showed you the different human input modes and how they affect
the behavior of the human-in-the-loop component.

The next chapter will be all about code executor -- the most powerful
component second only to LLMs.